# "Understanding The Data -- Kovaaks Aim Trainer"
> "A Deep Dive into the automatically saved csvs from Kovaaks Aim Trainer"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Josh Prier
- categories: [Kovaaks, Understanding The Data, Data Science]

# Kovaaks
Kovaaks is a "game" that allows to directly practice mouse control in 3d fps games. There are hundreds of different mini-games to practice with, each having a different focus. 

This [guide](https://www.dropbox.com/s/vaba3potfhf9jy1/KovaaK%20aim%20workout%20routines.pdf?dl=0) is best for getting perspective or understanding how and why to use Kovaaks


# File Names
Kovaaks has a great feature in which it saves every mini-game's stats to a csv. 
The format of the csv's names is as follows
```
<scenario name> - <Challenge or Freeplay> - YYYY.MM.DD-HH.MM.SS Stats.csv
```
Example:
```
Tile Frenzy - Challenge - 2020.12.14-08.46.00 Stats.csv
```

Some of the scenario names also have dashes so just checking against the first dash will not work
```
Tile Frenzy - Strafing - 03 - Challenge - 2020.12.14-08.34.31 Stats.csv
```

# The Data

Each file has 4 parts:
* List of all Kills
* Weapon, shots, hits, damage done, damage possible
* Overall Stats and info 
* Info about settings (input lag, fps, sens, FOV, etc)

In [79]:
#collapse-hide
import pandas as pd
from urllib.request import urlopen
from io import StringIO

Each part of the data has different formats and headers.
Here are the Headers/keys in python

In [72]:
#collapse-show
keys_kills=["Kill #","Timestamp","Bot","Weapon","TTK","Shots","Hits","Accuracy","Damage Done","Damage Possible","Efficiency","Cheated"]
keys_weapon=["Weapon","Shots","Hits","Damage Done","Damage Possible"]
keys_info=[]
keys_settings=[]

In [94]:
#collapse-show

# Current online directory for my stats 
stat_dir = "https://jprier.github.io/stats/"
stat_filenames_url = "https://jprier.github.io/stats/filenames.txt"

stat_filenames = urlopen(stat_filenames_url).read().decode('utf-8').split('\n')

kills = ""
weapon = ""
info = ""

for filename in stat_filenames:
    # TODO: parse filename for challenge name and date
    try:
        filename = filename.replace(' ', '%20')
        file = urlopen(stat_dir + filename).read().decode('utf-8').split('\n\n')

        # TODO: Add challenge name and date to each as columns 
        if kills == "":
            kills = file[0].split('\n')[0]
        file[0] = '\n'.join(file[0].split('\n')[1:])
        kills = kills + '\n' + file[0]

        # file[1] --> df_weapon
        # file[2,3] --> df_info
    except Exception as err:
        print(err)
    
df_kills = pd.read_csv(StringIO(kills), sep=",")

with pd.option_context('display.max_rows', 10, 'display.max_columns', None):
    display(df_kills)

HTTP Error 403: Forbidden


,Kill #,Timestamp,Bot,Weapon,TTK,Shots,Hits,Accuracy,Damage Done,Damage Possible,Efficiency,Cheated
0,1,14:19:43.846,target,pistol,0s,1.0,1.0,1.0,100.0,400.0,0.250,False
1,2,14:19:45.497,target,pistol,0.307s,2.0,1.0,0.5,100.0,800.0,0.125,False
2,3,14:19:46.148,target,pistol,0s,1.0,1.0,1.0,100.0,400.0,0.250,False
3,4,14:19:46.784,target,pistol,0s,1.0,1.0,1.0,100.0,400.0,0.250,False
4,5,14:19:47.384,target,pistol,0s,1.0,1.0,1.0,100.0,400.0,0.250,False
...,...,...,...,...,...,...,...,...,...,...,...,...
16781,87,16:47:47.289,Shinji small Bot,pistol,0s,1.0,1.0,1.0,2000.0,2000.0,1.000,False
16782,88,16:47:47.834,Shinji Bot,pistol,0s,1.0,1.0,1.0,2000.0,2000.0,1.000,False
16783,89,16:47:48.056,Shinji Bot,pistol,0s,1.0,1.0,1.0,2000.0,2000.0,1.000,False
16784,90,16:47:48.635,Shinji Bot,pistol,0s,1.0,1.0,1.0,2000.0,2000.0,1.000,False
